In [11]:
## pip install moviepy==1.0.0

import os
import librosa
import pandas as pd
from moviepy.editor import *
from tqdm import tqdm_notebook
import numpy as np
import re

In [12]:
#from utils import make_dir
def make_dir(dir_):
    if not os.path.isdir(dir_):
        os.mkdir(dir_)

In [13]:
background_dir = './data/audio_label_clip/background'  # 배경 디렉터리 경로
os.makedirs(background_dir, exist_ok=True)  # 배경 디렉터리 생성

activate_dir = './data/audio_label_clip/activate'  # 배경 디렉터리 경로
os.makedirs(activate_dir, exist_ok=True)  # 배경 디렉터리 생성

negative_dir = './data/audio_label_clip/negative'  # 배경 디렉터리 경로
os.makedirs(negative_dir, exist_ok=True)  # 배경 디렉터리 생성

In [14]:
audio_label_clip_dir = os.path.join('.', 'data', 'audio_label_clip')
make_dir(audio_label_clip_dir)

In [15]:
def save(audios, label):
    data = pd.DataFrame({
        'audios':audios, 'label':label })
    data.to_csv(audio_label_clip_dir+'/audio_label_clip.csv', index=False, encoding='euc-kr')

In [16]:
## 레이블링 데이터 위치 : ./data/label
files = []
for file in os.listdir(os.path.join('.', 'data', 'label')):
    _, file_id = os.path.split(file)
    if file_id.endswith('txt'):
        name = file_id.split(".")[0]
        if(name not in files):
            files.append(name)
print(files)

background_dir = './data/audio_label_clip/background'  # 배경 디렉터리 경로
os.makedirs(background_dir, exist_ok=True)  # 배경 디렉터리 생성

activate_dir = './data/audio_label_clip/activate'  # 배경 디렉터리 경로
os.makedirs(activate_dir, exist_ok=True)  # 배경 디렉터리 생성

negative_dir = './data/audio_label_clip/negative'  # 배경 디렉터리 경로
os.makedirs(negative_dir, exist_ok=True)  # 배경 디렉터리 생성print(files)


['2-6xb9sJT9s_0', '2-6xb9sJT9s_1', '2-6xb9sJT9s_2', '2-6xb9sJT9s_3', '2-6xb9sJT9s_4', '2-6xb9sJT9s_5', '2-6xb9sJT9s_6', '2-6xb9sJT9s_7', '2-6xb9sJT9s_8', '6io5HgRyGDU_0', '6io5HgRyGDU_1', '6io5HgRyGDU_2', '6io5HgRyGDU_3', '6io5HgRyGDU_4', '80LNjK_RQT4_0', '80LNjK_RQT4_1', '80LNjK_RQT4_2', '80LNjK_RQT4_3', '8CKFwKEDKII_0', '8CKFwKEDKII_1', '8CKFwKEDKII_2', '8CKFwKEDKII_3', '8CKFwKEDKII_4', '8CKFwKEDKII_5', '8CKFwKEDKII_6', 'aGY7n53AlB8_0', 'aGY7n53AlB8_1', 'aGY7n53AlB8_2', 'aGY7n53AlB8_3', 'aGY7n53AlB8_4', 'aGY7n53AlB8_5', 'aGY7n53AlB8_6', 'aGY7n53AlB8_7', 'aGY7n53AlB8_8', 'aiF9Lrgt5Xc_0', 'aiF9Lrgt5Xc_1', 'aiF9Lrgt5Xc_10', 'aiF9Lrgt5Xc_11', 'aiF9Lrgt5Xc_2', 'aiF9Lrgt5Xc_3', 'aiF9Lrgt5Xc_4', 'aiF9Lrgt5Xc_5', 'aiF9Lrgt5Xc_6', 'aiF9Lrgt5Xc_7', 'aiF9Lrgt5Xc_8', 'aiF9Lrgt5Xc_9', 'b270wrAtGAQ_0', 'b270wrAtGAQ_1', 'b270wrAtGAQ_10', 'b270wrAtGAQ_11', 'b270wrAtGAQ_2', 'b270wrAtGAQ_3', 'b270wrAtGAQ_4', 'b270wrAtGAQ_5', 'b270wrAtGAQ_6', 'b270wrAtGAQ_7', 'b270wrAtGAQ_8', 'b270wrAtGAQ_9', 'B3ah0WR4

In [6]:
background_dir = './data/audio_label_clip_for_test/background'  # 배경 디렉터리 경로
os.makedirs(background_dir, exist_ok=True)  # 배경 디렉터리 생성

activate_dir = './data/audio_label_clip_for_test/activate'  # 배경 디렉터리 경로
os.makedirs(activate_dir, exist_ok=True)  # 배경 디렉터리 생성

negative_dir = './data/audio_label_clip_for_test/negative'  # 배경 디렉터리 경로
os.makedirs(negative_dir, exist_ok=True)  # 배경 디렉터리 생성print(files)

In [17]:
audio = []
label = []
#라벨링
shi = 0
jon = 0
sha = 0
byoung = 0
sang = 0 
do = 0
mi = 0
gae = 0
jot = 0
background = 0
negative = 0
count = 0
x = 0

# 이제 욕 추가될때마다 여기 counter 디렉터리에 추가해주면 됨
counter = {
    10: shi,
    11: jon,
    12: sha,
    13: byoung,
    14: do,
    15: sang,
    16: mi,
    17: gae,
    18: jot,
    8: x,
    9: background,
    0: negative
}

for file in files:
    try: 
        print("file name:" + file)
        text = './data/label/' + file + ".txt"
        aud = './data/label/' + file + ".wav"
        # 정규표현식 패턴
        pattern = r"^(.*?)_\d+$"
        # 정규표현식 매칭
        match = re.match(pattern, file)
        raw_aud = './data/audio/' + str(match.group(1)) + ".wav"

        print("read new file")
        data = pd.read_csv(text, sep="\t", names=['start', 'end', 'label'])
        print(data)
        for i in range(len(data)):
            lab = data.label[i]
            print(data.start[i], data.end[i], data.label[i])
            try:
                start = float(data.start[i])
                end = float(data.end[i])
            except ValueError:
                continue
            if start >= end:
                continue
            if np.isnan(lab):
                continue
            full_audio = AudioFileClip(raw_aud)

            # 각 라벨 이름 정의
            lab_name = {
                10: "shi",
                11: "jon",
                12: "sha",
                13: "byoung",
                14: "do",
                15: "sang",
                16: "mi",
                17: "gae",
                18: "jot",
                8: "x",
                9: "background",
                0: "negative"
            }[lab]
            
            lab_counter = counter[lab]
            name = f"{lab_name}_{lab_counter}"
            counter[lab] += 1

            tmp_audio = full_audio.subclip(start, end)
            
            #라벨에 맞게 디렉터리 경로 초기화해주고
            if lab == 10 or lab == 11 or lab == 12 or lab == 13 or lab == 14 or lab == 15 or lab == 16 or lab == 17 or lab == 18:
                directory = activate_dir
            elif lab == 9:
                directory = background_dir
            elif lab == 0:
                directory = negative_dir
            else:
                continue
            # 라벨에 맞는 디렉터리에 저장
            name = os.path.join(directory, name + ".wav")
            try:
                tmp_audio.write_audiofile(name)
            except OSError:
                print('oserror')
                continue
                
            audio.append(name)
            label.append(int(lab))
            print('완료됬습니다.')
        count += 1
    except Exception as e:
        pass

save(audio, label)

file name:2-6xb9sJT9s_0
read new file
   start  end  label
0    0.0  1.2      0
0.0 1.2 0
MoviePy - Writing audio in ./data/audio_label_clip/negative\negative_0.wav


MoviePy - Done.
완료됬습니다.
file name:2-6xb9sJT9s_1
read new file
   start   end  label
0   2.68  3.57     10
2.68 3.57 10


MoviePy - Writing audio in ./data/audio_label_clip/activate\shi_0.wav


MoviePy - Done.
완료됬습니다.
file name:2-6xb9sJT9s_2
read new file
   start  end  label
0   3.74  4.7      0
3.74 4.7 0


MoviePy - Writing audio in ./data/audio_label_clip/negative\negative_1.wav


...

MoviePy - Done.
완료됬습니다.
file name:xH5ANOiuoZg_7
read new file
   start    end  label
0   14.7  16.46      0
14.7 16.46 0


MoviePy - Writing audio in ./data/audio_label_clip/negative\negative_288.wav


MoviePy - Done.
완료됬습니다.
file name:xH5ANOiuoZg_8
read new file
   start    end  label
0  16.46  19.15      0
16.46 19.15 0


MoviePy - Writing audio in ./data/audio_label_clip/negative\negative_289.wav


MoviePy - Done.
완료됬습니다.
